In [1]:
"""
Author: Yanzheng Wu(yanzheng.wu@tufts.edu)
Reference: NandaKishore Joshi - https://github.com/NandaKishoreJoshi/Reinforcement_Lerning
"""
import Gridworld as GW
import torch
import numpy as np
import matplotlib
from matplotlib import pylab as plt
import random

game = GW.Gridworld(size=4, mode="static")

l1 = 64
l2 = 150
l3 = 100
l4 = 4
gamma = 0.9
epsilon = 1.0

#NN model for the agent
model = torch.nn.Sequential(
    torch.nn.Linear(l1, l2),
    torch.nn.ReLU(),
    torch.nn.Linear(l2, l3),
    torch.nn.ReLU(),
    torch.nn.Linear(l3,l4)
)
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 1000
losses = []   
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}
for i in range(epochs):                                
    game = GW.Gridworld(size=4, mode='static')            
    state_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0               
    state1 = torch.from_numpy(state_).float()          
    status = 1                                         
    while(status == 1):                                
        qval = model(state1)                           
        qval_ = qval.data.numpy()
        if (random.random() < epsilon):                
            action_ = np.random.randint(0,4)
        else:
            action_ = np.argmax(qval_)
        
        action = action_set[action_]                   
        game.makeMove(action)                          
        state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
        state2 = torch.from_numpy(state2_).float()     
        reward = game.reward()
        with torch.no_grad():
            newQ = model(state2.reshape(1,64))
        maxQ = torch.max(newQ)                         
        if reward == -1:                               
            Y = reward + (gamma * maxQ)
        else:
            Y = reward
        Y = torch.Tensor([Y]).detach()
        X = qval.squeeze()[action_]                    
        loss = loss_fn(X, Y)
        optimizer.zero_grad()
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        state1 = state2
        if reward != -1:                               
            status = 0
    if epsilon > 0.1:                                  
        epsilon -= (1/epochs)